In [1]:
import torch
from d2l import torch as d2l

# torch.rand(size) # 均匀分布U(0, 1)
# torch.randn(size) # 高斯正太, random normal
# torch.normal(mean, std, size)
# torch.arange(start, end, step)
# torch.FloatTensor([]) 创建一个float32的张量

In [2]:
# 当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有3*h*w的形状。
# 我们将这个大小为3的轴称为通道（channel）维度。本节将更深入地研究具有多输入和多输出通道的卷积核。
# 为了加深理解，我们实现一下多输入通道互相关运算。 简而言之，我们所做的就是对每个通道执行互相关操作，然后将结果相加。

import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [3]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K), K.shape, X.shape

(tensor([[ 56.,  72.],
         [104., 120.]]),
 torch.Size([2, 2, 2]),
 torch.Size([2, 3, 3]))

In [4]:
# 在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。
# 直观地说，我们可以将每个通道看作对不同特征的响应。而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。
# 因此，多输出通道并不仅是学习多个单通道的检测器。
# 我们把通道的输入定义为 ci co，分别代表 input & output
# 我们可以为每个输出通道创建一个 ci * kh * kw 的卷积核张量。这样卷积核形状为 co * ci * kh * kw

##### 如下所示，我们实现一个计算多个通道的输出的互相关函数。

In [5]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
    # for K in k 其实是按照c_o在取出,那么取出的c_i对于X,k都是一样的
    # 每一组卷积核都和X的通道数量一样多

In [6]:
# torch.stack ： 在新的维度上，把多个张量堆叠. 默认 dim = 0
K = torch.stack((K, K + 1, K + 2), 0) # torch.stack((tensor), dim)
K, K.shape, X.shape

(tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]),
 torch.Size([3, 2, 2, 2]),
 torch.Size([2, 3, 3]))

In [7]:
# 下面，我们对输入张量X与卷积核张量K执行互相关运算。
# 现在的输出包含3个通道，第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致

In [1]:
corr2d_multi_in_out(X, K)
# 这里 X 的形状是 2, 3, 3, 2个通道, 形状3*3
# K 的形状是 3 * 2 * 2 * 2 , 3是输出的通道形状, 对于每一个输出通道，让 2组2*2卷积核对应上X的两组 3*3图像

NameError: name 'corr2d_multi_in_out' is not defined

In [ ]:
X₁ = 
| 1  | 2  | 3 |
| 4  | 5  | 6 |
| 7  | 8  | 9 |
X₂ = 
| 9  | 8  | 7 |
| 6  | 5  | 4 |
| 3  | 2  | 1 |
K = 
| 1  | 0 |，| 3  | 3 |，| 5  | 9 |
| 0  | 1 |，| 4  | 2 |，| 1  | 3 |

In [9]:
# 1*1 卷积层的意义可以是变化通道数量
# 本质就是对每一个像素位置，做一个全连接映射
# 输入 c_o , 输出 c_i , 卷积核 c_o * c_i * 1 * 1

In [11]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape # X 可能是 3 * 2 * 2的输入，三通道，每个通道2方针
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w)) # reshape 成 3 , 4
    K = K.reshape((c_o, c_i)) # 把 co * ci * 1 * 1简化
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [14]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

In [ ]:
# 没有assert报错就说明正确性